In [2]:
import pandas as pd
import sqlite3

In [3]:
pd.options.display.max_columns = None

In [4]:
conn = sqlite3.connect('sakila.db')

In [5]:
cursor = conn.cursor()
cursor.execute('''
SELECT * FROM sqlite_master WHERE type = 'table'
''')
data = cursor.fetchall()
columns = [col[0] for col in cursor.description]
cursor.close()

In [6]:
data_df = pd.DataFrame(data, columns=columns)
data_df

,type,name,tbl_name,rootpage,sql
0,table,actor,actor,2,"CREATE TABLE ""actor"" (\n""index"" INTEGER,\n ""a..."
1,table,store,store,7,"CREATE TABLE ""store"" (\n""index"" INTEGER,\n ""s..."
2,table,address,address,9,"CREATE TABLE ""address"" (\n""index"" INTEGER,\n ..."
3,table,category,category,26,"CREATE TABLE ""category"" (\n""index"" INTEGER,\n ..."
4,table,city,city,28,"CREATE TABLE ""city"" (\n""index"" INTEGER,\n ""ci..."
5,table,country,country,39,"CREATE TABLE ""country"" (\n""index"" INTEGER,\n ..."
6,table,customer,customer,43,"CREATE TABLE ""customer"" (\n""index"" INTEGER,\n ..."
7,table,film_actor,film_actor,63,"CREATE TABLE ""film_actor"" (\n""index"" INTEGER,\..."
8,table,film_category,film_category,126,"CREATE TABLE ""film_category"" (\n""index"" INTEGE..."
9,table,inventory,inventory,140,"CREATE TABLE ""inventory"" (\n""index"" INTEGER,\n..."


In [7]:
pd.read_sql_query(sql = '''
SELECT * FROM actor;
''', con = conn)

,index,actor_id,first_name,last_name,last_update
0,0,1,Penelope,Guiness,2013-05-26 14:47:57.620000
1,1,2,Nick,Wahlberg,2013-05-26 14:47:57.620000
2,2,3,Ed,Chase,2013-05-26 14:47:57.620000
3,3,4,Jennifer,Davis,2013-05-26 14:47:57.620000
4,4,5,Johnny,Lollobrigida,2013-05-26 14:47:57.620000
...,...,...,...,...,...
195,195,196,Bela,Walken,2013-05-26 14:47:57.620000
196,196,197,Reese,West,2013-05-26 14:47:57.620000
197,197,198,Mary,Keitel,2013-05-26 14:47:57.620000
198,198,199,Julia,Fawcett,2013-05-26 14:47:57.620000


Question 1:

Show amount of rentals during days in week.

In [15]:
# sqlite 3 version

pd.read_sql_query(sql = '''
SELECT rental_day_of_week, daily_sum_amount, SUM(daily_sum_amount) OVER (ORDER BY rental_date) AS cumulative_sum_amount
FROM (SELECT SUM(p.amount) AS daily_sum_amount, strftime('%w', r.rental_date) AS rental_date,
case cast (strftime('%w', r.rental_date) as integer)
  WHEN 0 THEN 'Sunday'
  WHEN 1 THEN 'Monday'
  WHEN 2 THEN 'Tuesday'
  WHEN 3 THEN 'Wednesday'
  WHEN 4 THEN 'Thursday'
  WHEN 5 THEN 'Friday'
  ELSE 'Saturday' END AS rental_day_of_week
FROM film AS f
JOIN inventory AS i
ON i.film_id = f.film_id
JOIN rental AS r
ON i.inventory_id = r.inventory_id
JOIN payment AS p
ON p.rental_id = r.rental_id
GROUP BY 2,3 
ORDER BY 2) t1;
''', con = conn)

,rental_day_of_week,daily_sum_amount,cumulative_sum_amount
0,Sunday,8900.74,8900.74
1,Monday,8555.51,17456.25
2,Tuesday,9296.57,26752.82
3,Wednesday,8495.51,35248.33
4,Thursday,8423.09,43671.42
5,Friday,8715.34,52386.76
6,Saturday,8925.28,61312.04


In [14]:
pd.read_sql_query(sql = '''
SELECT rental_day_of_week, daily_sum_amount, SUM(daily_sum_amount) OVER (ORDER BY rental_date) AS cumulative_sum_amount
FROM (SELECT SUM(p.amount) AS daily_sum_amount, strftime('%w', r.rental_date) AS rental_date,
case cast (strftime('%w', r.rental_date) as integer)
  WHEN 0 THEN 'Sunday'
  WHEN 1 THEN 'Monday'
  WHEN 2 THEN 'Tuesday'
  WHEN 3 THEN 'Wednesday'
  WHEN 4 THEN 'Thursday'
  WHEN 5 THEN 'Friday'
  ELSE 'Saturday' END AS rental_day_of_week
FROM film AS f
JOIN inventory AS i
ON i.film_id = f.film_id
JOIN rental AS r
ON i.inventory_id = r.inventory_id
JOIN payment AS p
ON p.rental_id = r.rental_id
GROUP BY 2,3 
ORDER BY 2) t1;
''', con = conn).to_excel('aaa.xlsx')

Solution:

SELECT rental_day_of_week, daily_sum_amount, SUM(daily_sum_amount) OVER (ORDER BY rental_date) AS cumulative_sum_amount
FROM (SELECT SUM(p.amount) AS daily_sum_amount, DATE_PART('dow', r.rental_date) AS rental_date,
CASE DATE_PART('dow', r.rental_date)
  WHEN 0 THEN 'Sunday'
  WHEN 1 THEN 'Monday'
  WHEN 2 THEN 'Tuesday'
  WHEN 3 THEN 'Wednesday'
  WHEN 4 THEN 'Thursday'
  WHEN 5 THEN 'Friday'
  ELSE 'Saturday' END AS rental_day_of_week
FROM film AS f
JOIN inventory AS i
ON i.film_id = f.film_id
JOIN rental AS r
ON i.inventory_id = r.inventory_id
JOIN payment AS p
ON p.rental_id = r.rental_id
GROUP BY 2,3 
ORDER BY 2) t1;

In [10]:
pd.read_sql_query(sql = '''
SELECT SUM(amount) FROM payment;
''', con = conn)

,SUM(amount)
0,61312.04
